# CIFAR‑10 Image Classification: Graph‑based vs. CNN Baseline

In this notebook we compare two approaches for CIFAR‑10 image classification:
1. **Graph‑based Model:** Each 32×32 CIFAR‑10 image is divided into 16 patches (a 4×4 grid, each patch is 8×8). These patches become node features, and adjacent patches are connected to form a graph. We then process the graph using our unified CNN‑GNN model (which integrates a modern CNN encoder with dropout, batch normalization, and residual connections into a GNN).
2. **CNN Baseline:** A conventional convolutional neural network for CIFAR‑10 classification.

We train both models and compare their learning curves and performance.


In [6]:
# Import libraries and set up device
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# TGraphX imports
from models.cnn_gnn_model import CNN_GNN_Model
from core.graph import Graph, GraphBatch
from core.dataloader import GraphDataset, GraphDataLoader
from core.utils import get_device

device = get_device()
print("Using device:", device)


Using device: cuda


## Custom Dataset: CIFAR‑10 Graph Dataset

We define a dataset that loads CIFAR‑10 images and converts each image into a graph using the helper above.


In [7]:
from torchvision.datasets import CIFAR10

class CIFAR10GraphDataset(torch.utils.data.Dataset):
    def __init__(self, root, train=True, transform=None, grid_size=4):
        self.dataset = CIFAR10(root=root, train=train, download=True, transform=transform)
        self.grid_size = grid_size

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        img, label = self.dataset[idx]  # img is a tensor, label is an int
        # Convert image (3, 32, 32) into graph (node_features and edge_index)
        node_features, edge_index = image_to_graph(img, grid_size=self.grid_size)
        from core.graph import Graph
        graph = Graph(node_features, edge_index)
        return graph, label

# Define transform to convert PIL image to tensor
transform = transforms.Compose([
    transforms.ToTensor(),
    # Optionally add normalization
])


## DataLoader Setup

We use the custom `GraphDataLoader` from TGraphX with a collate function that batches (graph, label) tuples.


In [8]:
def graph_label_collate_fn(batch):
    """Batch a list of (Graph, label) tuples."""
    graphs, labels = zip(*batch)
    from core.graph import GraphBatch
    return GraphBatch(graphs), torch.tensor(labels, dtype=torch.long)

train_dataset = CIFAR10GraphDataset(root='./data', train=True, transform=transform, grid_size=4)
test_dataset = CIFAR10GraphDataset(root='./data', train=False, transform=transform, grid_size=4)

from core.dataloader import GraphDataLoader
train_loader = GraphDataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=graph_label_collate_fn)
test_loader  = GraphDataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=graph_label_collate_fn)


## Helper Functions: Converting an Image to a Graph

Each CIFAR‑10 image (3×32×32) is split into a 4×4 grid of patches (each 3×8×8).
Edges are created between patches that are adjacent (right and bottom neighbors).


In [16]:
import torch
import random

def image_to_graph(img, grid_size=4, return_edge_features=False):
    """
    Converts an image tensor (C, H, W) into a graph.
    Divides the image into a regular grid of patches.

    Patches are taken in a sliding window fashion with no randomness.
    (For example, each 8x8 patch is taken with a stride of 8, covering the whole image.)

    Args:
        img (Tensor): Input image tensor of shape (C, H, W)
        grid_size (int): Number of patches per side. For CIFAR-10 (32x32), grid_size=4 gives 8x8 patches.
        return_edge_features (bool): If True, also compute and return edge features.

    Returns:
        node_features: Tensor of shape (grid_size*grid_size, C, patch_H, patch_W)
        edge_index: Tensor of shape [2, E]
        (If return_edge_features is True, also returns edge_features, otherwise not.)
    """
    C, H, W = img.shape
    patch_H = H // grid_size
    patch_W = W // grid_size
    nodes = []
    # Create patches using a sliding window that moves one patch at a time (non-overlapping)
    for i in range(grid_size):
        for j in range(grid_size):
            patch = img[:, i*patch_H:(i+1)*patch_H, j*patch_W:(j+1)*patch_W]
            nodes.append(patch)
    node_features = torch.stack(nodes, dim=0)

    edges = []
    def node_index(i, j):
        return i * grid_size + j

    # Create structured edges between adjacent patches (right and bottom neighbors)
    for i in range(grid_size):
        for j in range(grid_size):
            idx = node_index(i, j)
            if j < grid_size - 1:
                edges.append([idx, node_index(i, j+1)])
                edges.append([node_index(i, j+1), idx])
            if i < grid_size - 1:
                edges.append([idx, node_index(i+1, j)])
                edges.append([node_index(i+1, j), idx])

    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()

    if return_edge_features:
        # Example: set edge_features as ones of shape (E, 1) (or modify as needed)
        edge_features = torch.ones(edge_index.shape[1], 1)
        return node_features, edge_index, edge_features
    else:
        return node_features, edge_index

# Example usage in your dataset (unchanged backend):
# In your CIFAR10GraphDataset __getitem__, you can still call:
# node_features, edge_index = image_to_graph(img, grid_size=self.grid_size)






# Test the helper with a sample image from the dataset
sample_graph, sample_label = train_dataset[0]
print("Graph Sample Summary:")
print("-" * 40)
print(f"Label: {sample_label}")
print(f"Node features shape: {sample_graph.node_features.shape}")
print(f"Edge index shape: {sample_graph.edge_index.shape}")
print("First 10 edges:")
print(sample_graph.edge_index[:, :10])
print("-" * 40)


Graph Sample Summary:
----------------------------------------
Label: 6
Node features shape: torch.Size([16, 3, 8, 8])
Edge index shape: torch.Size([2, 48])
First 10 edges:
tensor([[0, 1, 0, 4, 1, 2, 1, 5, 2, 3],
        [1, 0, 4, 0, 2, 1, 5, 1, 3, 2]])
----------------------------------------


## Model 1: Graph-based Classifier using TGraphX

Our graph-based model uses node features from image patches. For each graph:
- **Input shape:** (3, 8, 8) per node.
- **Hidden shape:** We choose (16, 8, 8).
- **Pooling:** Mean pooling over nodes.
- **Classifier:** Outputs 10 classes.


In [17]:
# Simplified configuration dictionary (≈30% lighter complexity)
config = {
    "cnn_params": {
         "in_channels": 3,              # RGB input
         "out_features": 64,            # Output feature map depth
         "num_layers": 3,               # Reduced total convolutional layers (from 5 to 3)
         "hidden_channels": 64,         # Intermediate channels remain the same
         "dropout_prob": 0.3,           # Dropout probability for regularization
         "use_batchnorm": True,         # Use BatchNorm in CNN layers
         "use_residual": True,          # Enable residual connections in the CNN encoder
         "pool_layers": 1,              # Fewer pooling layers (from 2 to 1)
         "debug": False,                # Disable debug printing
         "return_feature_map": True     # Return a spatial feature map (do not flatten)
    },
    "use_preencoder": True,              # Enable the optional pre-encoder stage
    "pretrained_resnet": True,           # Use pretrained ResNet-18 weights for the pre-encoder
    "preencoder_params": {
         "in_channels": 3,             # RGB input for the pre-encoder
         "out_channels": 32,           # Output channels from the pre-encoder
         "hidden_channels": 32         # Hidden channels in the pre-encoder (if not pretrained)
    },
    "gnn_in_dim": (64, 5, 5),            # Must match CNN encoder output shape
    "gnn_hidden_dim": (128, 5, 5),         # Hidden state for GNN layers (spatial: channels, H, W)
    "num_classes": 10,                   # e.g. 10 classes for CIFAR-10
    "num_gnn_layers": 3,                 # Reduced GNN layers (from 4 to 3)
    "gnn_dropout": 0.3,                  # Dropout rate for GNN layers
    "residual": True,                    # Enable residual connections in the GNN layers
    "aggregator_params": {
         "num_layers": 2,               # Fewer aggregator layers (from 4 to 2)
         "dropout_prob": 0.3,           # Dropout probability remains
         "use_batchnorm": True          # Use BatchNorm in the aggregator CNN layers
    },
    "skip_cnn_to_classifier": True       # Extra skip connection from CNN encoder directly to classifier
}

from models.pre_encoder import PreEncoder  # Import the PreEncoder module
from models.cnn_gnn_model import CNN_GNN_Model  # Import the unified CNN-GNN model
from core.utils import get_device             # Helper function to get the available device

# Instantiate the pre-encoder if enabled.
if config["use_preencoder"]:
    pre_encoder = PreEncoder(
         in_channels=config["preencoder_params"]["in_channels"],
         out_channels=config["preencoder_params"]["out_channels"],
         use_pretrained=config["pretrained_resnet"],
         custom_params=config.get("preencoder_params")
    )
else:
    pre_encoder = None  # Raw input is passed directly to the CNN encoder

# Create the unified CNN-GNN model using the simplified configuration.
model = CNN_GNN_Model(
    cnn_params=config["cnn_params"],
    gnn_in_dim=config["gnn_in_dim"],
    gnn_hidden_dim=config["gnn_hidden_dim"],
    num_classes=config["num_classes"],
    num_gnn_layers=config["num_gnn_layers"],
    gnn_dropout=config["gnn_dropout"],
    residual=config["residual"],
    aggregator_params=config["aggregator_params"],
    pre_encoder=pre_encoder
)

# Get available device (GPU if available, else CPU)
device = get_device()

# Move the model to the chosen device.
model.to(device)


Loading pretrained ResNet-18 weights...
ResNet-18 model loaded. Checking model parameters:
conv1.weight: torch.Size([64, 3, 7, 7])
bn1.weight: torch.Size([64])
bn1.bias: torch.Size([64])
layer1.0.conv1.weight: torch.Size([64, 64, 3, 3])
layer1.0.bn1.weight: torch.Size([64])
layer1.0.bn1.bias: torch.Size([64])
layer1.0.conv2.weight: torch.Size([64, 64, 3, 3])
layer1.0.bn2.weight: torch.Size([64])
layer1.0.bn2.bias: torch.Size([64])
layer1.1.conv1.weight: torch.Size([64, 64, 3, 3])
layer1.1.bn1.weight: torch.Size([64])
layer1.1.bn1.bias: torch.Size([64])
layer1.1.conv2.weight: torch.Size([64, 64, 3, 3])
layer1.1.bn2.weight: torch.Size([64])
layer1.1.bn2.bias: torch.Size([64])
layer2.0.conv1.weight: torch.Size([128, 64, 3, 3])
layer2.0.bn1.weight: torch.Size([128])
layer2.0.bn1.bias: torch.Size([128])
layer2.0.conv2.weight: torch.Size([128, 128, 3, 3])
layer2.0.bn2.weight: torch.Size([128])
layer2.0.bn2.bias: torch.Size([128])
layer2.0.downsample.0.weight: torch.Size([128, 64, 1, 1])
laye

CNN_GNN_Model(
  (encoder): CNNEncoder(
    (pre_encoder): PreEncoder(
      (features): Sequential(
        (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (4): Sequential(
          (0): BasicBlock(
            (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu): ReLU(inplace=True)
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
          (1): BasicBlock(
            (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1),

## Model 2: CNN Baseline for CIFAR‑10

Below is a simple CNN architecture for CIFAR‑10 classification.


In [18]:
# Define a simple CNN baseline for CIFAR‑10 classification.
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),  # 32 -> 16
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)   # 16 -> 8
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 8 * 8, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

cnn_model = SimpleCNN(num_classes=10)
cnn_model.to(device)


SimpleCNN(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=4096, out_features=128, bias=True)
    (2): ReLU(inplace=True)
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=128, out_features=10, bias=True)
  )
)

In [19]:
import torch.optim as optim

graph_optimizer = optim.Adam(model.parameters(), lr=0.001)
cnn_optimizer   = optim.Adam(cnn_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()


In [20]:
from torchvision.datasets import CIFAR10

# Create DataLoaders for the CNN baseline using the original CIFAR‑10 images.
train_cifar = CIFAR10(root='./data', train=True, download=True, transform=transform)
test_cifar  = CIFAR10(root='./data', train=False, download=True, transform=transform)

cnn_train_loader = torch.utils.data.DataLoader(train_cifar, batch_size=32, shuffle=True)
cnn_test_loader  = torch.utils.data.DataLoader(test_cifar, batch_size=32, shuffle=False)


## Training Loop

We train both models on the CIFAR‑10 training set for a few epochs and compare their performance.
For the graph-based model, the input graphs are converted from image patches, while the CNN uses the original images.


In [21]:
from sklearn.metrics import balanced_accuracy_score, f1_score

def evaluate_model(model, loader, device, is_cnn=False):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for data in loader:
            if is_cnn:
                images, labels = data
                images = images.to(device)
                outputs = model(images)
            else:
                graphs, labels = data
                graphs = graphs.to(device)
                outputs = model(graphs.node_features, graphs.edge_index, graphs.edge_features, graphs.batch)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())
    ba = balanced_accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    return ba, f1

num_epochs = 100
graph_train_losses = []
graph_ba_list = []
graph_f1_list = []
cnn_train_losses = []
cnn_ba_list = []
cnn_f1_list = []

print("\nTraining Both Models Simultaneously:\n")
header = "{:<6s} {:<14s} {:<14s} {:<14s} {:<14s} {:<14s} {:<14s}".format(
    "Epoch", "TGraphX Loss", "TGraphX BA", "TGraphX F1", "CNN Loss", "CNN BA", "CNN F1"
)
print(header)
print("-" * len(header))

for epoch in range(1, num_epochs + 1):
    # --- Train Graph-based Model ---
    model.train()
    total_graph_loss = 0.0
    total_graph_samples = 0
    graph_nan_detected = False  # flag to mark if NaN is detected in graph training
    for batch, labels in train_loader:
        # Optionally, you could print the batch shape for the first batch
        #if total_graph_samples == 0:
        #    print("Batch node_features shape:", batch.node_features.shape)
        batch = batch.to(device)
        labels = labels.to(device)
        graph_optimizer.zero_grad()

        outputs = model(batch.node_features, batch.edge_index, batch.edge_features, batch.batch)
        if torch.isnan(outputs).any():
            print("Model output contains NaN!")
            print("Output stats: mean={:.4f}, std={:.4f}, min={:.4f}, max={:.4f}".format(
                outputs.mean().item(), outputs.std().item(), outputs.min().item(), outputs.max().item()
            ))
            graph_nan_detected = True
            break

        loss = criterion(outputs, labels)
        if torch.isnan(loss):
            print("NaN loss detected in Graph model!")
            graph_nan_detected = True
            break

        loss.backward()
        graph_optimizer.step()
        total_graph_loss += loss.item() * labels.size(0)
        total_graph_samples += labels.size(0)

    if graph_nan_detected or total_graph_samples == 0:
        epoch_graph_loss = float('nan')
        graph_train_losses.append(epoch_graph_loss)
        print(f"Epoch {epoch}: Graph training encountered NaN values.")
    else:
        epoch_graph_loss = total_graph_loss / total_graph_samples
        graph_train_losses.append(epoch_graph_loss)

    # --- Train CNN Baseline ---
    cnn_model.train()
    total_cnn_loss = 0.0
    total_cnn_samples = 0
    for images, labels in cnn_train_loader:
        cnn_optimizer.zero_grad()
        images = images.to(device)
        labels = labels.to(device)
        outputs = cnn_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        cnn_optimizer.step()
        total_cnn_loss += loss.item() * labels.size(0)
        total_cnn_samples += labels.size(0)
    epoch_cnn_loss = total_cnn_loss / total_cnn_samples
    cnn_train_losses.append(epoch_cnn_loss)

    # --- Evaluate Models ---
    graph_ba, graph_f1 = evaluate_model(model, test_loader, device, is_cnn=False)
    cnn_ba, cnn_f1 = evaluate_model(cnn_model, cnn_test_loader, device, is_cnn=True)

    # Store metrics for plotting
    graph_ba_list.append(graph_ba)
    graph_f1_list.append(graph_f1)
    cnn_ba_list.append(cnn_ba)
    cnn_f1_list.append(cnn_f1)

    print("{:<6d} {:<14.4f} {:<14.4f} {:<14.4f} {:<14.4f} {:<14.4f} {:<14.4f}".format(
        epoch, epoch_graph_loss, graph_ba, graph_f1, epoch_cnn_loss, cnn_ba, cnn_f1
    ))

print("\nTraining complete!")



Training Both Models Simultaneously:

Epoch  TGraphX Loss   TGraphX BA     TGraphX F1     CNN Loss       CNN BA         CNN F1        
------------------------------------------------------------------------------------------------
1      1.8473         0.3078         0.2795         1.6381         0.5573         0.5533        
2      1.7169         0.3748         0.3654         1.4064         0.5279         0.5152        
3      1.6644         0.3765         0.3654         1.3201         0.5486         0.5358        
4      1.6132         0.4248         0.4233         1.2548         0.5785         0.5695        
5      1.5516         0.3933         0.3741         1.2057         0.5639         0.5631        
6      1.5035         0.4559         0.4321         1.1512         0.6197         0.6191        
7      1.4711         0.4562         0.4449         1.1057         0.6797         0.6861        
8      1.4455         0.4650         0.4420         1.0684         0.6629         0.6604

## Learning Curve Comparison

Below we plot the training loss curves for both models.


In [22]:
import matplotlib.pyplot as plt

# Assume these lists are populated during training:
# graph_train_losses, graph_ba_list, graph_f1_list,
# cnn_train_losses, cnn_ba_list, cnn_f1_list

epochs = range(1, num_epochs + 1)

fig, axs = plt.subplots(1, 3, figsize=(18, 5), sharex=True)

# Loss plot: Compare TGraphX and CNN losses in the same plot
axs[0].plot(epochs, graph_train_losses, color='tab:blue', label='TGraphX Loss')
axs[0].plot(epochs, cnn_train_losses, color='tab:orange', label='CNN Loss')
axs[0].set_title("Loss")
axs[0].set_ylabel("Loss")
axs[0].set_xlabel("Epoch")
axs[0].grid(True)
axs[0].legend()

# Balanced Accuracy plot: Compare TGraphX and CNN BA in the same plot
axs[1].plot(epochs, graph_ba_list, color='tab:blue', label='TGraphX BA')
axs[1].plot(epochs, cnn_ba_list, color='tab:orange', label='CNN BA')
axs[1].set_title("Balanced Accuracy")
axs[1].set_ylabel("Balanced Accuracy")
axs[1].set_xlabel("Epoch")
axs[1].grid(True)
axs[1].legend()

# F1 Score plot: Compare TGraphX and CNN F1 in the same plot
axs[2].plot(epochs, graph_f1_list, color='tab:blue', label='TGraphX F1')
axs[2].plot(epochs, cnn_f1_list, color='tab:orange', label='CNN F1')
axs[2].set_title("F1 Score")
axs[2].set_ylabel("F1 Score")
axs[2].set_xlabel("Epoch")
axs[2].grid(True)
axs[2].legend()

plt.tight_layout()
plt.show()
